https://mols2grid.readthedocs.io/en/latest/notebooks/quickstart.html

Download test sdf file

In [1]:
# import urllib.request

# url = 'https://raw.githubusercontent.com/rdkit/rdkit/master/Docs/Book/data/solubility.test.sdf'
# urllib.request.urlretrieve(url, 'solubility.test.sdf')

('solubility.test.sdf', <http.client.HTTPMessage at 0x10484e3d0>)

In [1]:
import mols2grid
SDF_FILE = "/Users/lpc_0066/Desktop/MI/molecule_visualization/solubility.test.sdf"
mols2grid.display(SDF_FILE)

MolGridWidget()

In [9]:
from rdkit import Chem

# Load the SDF file
suppl = Chem.SDMolSupplier(SDF_FILE)

# Check the fields for the first molecule
mol = suppl[0]
if mol is not None:
    fields = mol.GetPropNames()
    for field in fields:
        print(f"{field}: {mol.GetProp(field)}")
else:
    print("No molecules found in the SDF file.")

ID: 5
NAME: 3-methylpentane
SOL: -3.68
SMILES: CCC(C)CC
SOL_classification: (A) low


USe Dataframe as input

In [6]:
import pandas as pd
from rdkit.Chem import PandasTools
df = pd.read_csv("/Users/lpc_0066/Desktop/MI/molecular_similarity/VDss_dataset_uen.csv")
PandasTools.AddMoleculeColumnToFrame(frame=df, smilesCol="SMILES")
mols2grid.display(df, mol_col="ROMol")

MolGridWidget()

Customization (CSV->customized SDF->display)

In [11]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, PandasTools
df = pd.read_csv("/Users/lpc_0066/Desktop/MI/molecular_similarity/VDss_dataset_uen.csv")
df.columns

Index(['SMILES', 'MW', 'logP', 'VDss(L/kg)', 'logVDss'], dtype='object')

In [12]:
mols = [Chem.MolFromSmiles(smi) for smi in df.SMILES]

for mol in mols:
    Chem.AddHs(mol)
    AllChem.Compute2DCoords(mol)

In [14]:
# Make SDF file with selected properties
writer = Chem.SDWriter("customized_VDss_dataset.sdf")

for (mol, mw, logp) in zip(mols, df.MW, df.logP):
    mol.SetProp("molecular_weight", str(mw))
    mol.SetProp("LogP", str(logp))
    writer.write(mol)
writer.close()

(Optional) If you need some spice!

In [19]:
# css_style = """
# /* rainbow background */
# .m2g-cell {
#     background: linear-gradient(124deg, #ff2400, #e81d1d, #e8b71d, #e3e81d, #1de840, #1ddde8, #2b1de8, #dd00f3, #dd00f3);
#     background-size: 500% 500%;
#     -webkit-animation: rainbow 10s ease infinite;
#     animation: rainbow 10s ease infinite;
# }
# .m2g-cell:hover {
#     border-color: red !important;
# }
# /* rainbow font color */
# .data {
#     font-weight: bold;
#     -webkit-text-stroke: 1px black;
#     background: linear-gradient(to right, #ef5350, #f48fb1, #7e57c2, #2196f3, #26c6da, #43a047, #eeff41, #f9a825, #ff5722);
#     -webkit-background-clip: text;
#     color: transparent;
# }
# /* background animation */
# @-webkit-keyframes rainbow {
#     0% {background-position: 0% 50%}
#     50% {background-position: 100% 50%}
#     100% {background-position: 0% 50%}
# }
# @keyframes rainbow {
#     0% {background-position: 0% 50%}
#     50% {background-position: 100% 50%}
#     100% {background-position: 0% 50%}
# }
# """

In [21]:
CUSTOMIZED_SDF_FILE = "/Users/lpc_0066/Desktop/MI/molecule_visualization/customized_VDss_dataset.sdf"

mols2grid.display(
    CUSTOMIZED_SDF_FILE,
    # rename fields for the output document
    rename={"molecular_weight": "MW", "LogP": "Solubility"},

    # set what's displayed on the grid
    subset=["MW", "Solubility"],
    
    # set what's displayed on the hover tooltip (For string, displayed at top, for number, displayed at bottom)
    tooltip=["Solubility"],
    
    # style for the grid labels and tooltips
    style={
        "Solubility": lambda x: "color: red; font-weight: bold;" if x < -3 else ""
    },
    # change the precision and format (or other transformations)
    transform={"Solubility": lambda x: round(x, 2)},
    
    # sort the grid in a different order by default
    # sort_by="Name",
    
    # molecule drawing parameters
    fixedBondLength=25,
    clearBackground=False,

    # Rock and roll!
    # custom_css=css_style,
)

[15:22:35] Warning: ambiguous stereochemistry - overlapping neighbors  - at atom 15 ignored
[15:22:37] Warning: ambiguous stereochemistry - overlapping neighbors  - at atom 15 ignored
[15:22:37] Warning: ambiguous stereochemistry - zero final chiral volume - at atom 18 ignored
[15:22:37] Warning: ambiguous stereochemistry - linear bond arrangement - at atom 21 ignored
[15:22:38] Warning: ambiguous stereochemistry - zero final chiral volume - at atom 19 ignored
[15:22:38] Warning: ambiguous stereochemistry - linear bond arrangement - at atom 13 ignored
[15:22:38] Warning: ambiguous stereochemistry - linear bond arrangement - at atom 14 ignored
[15:22:38] Warning: ambiguous stereochemistry - linear bond arrangement - at atom 9 ignored
[15:22:38] Warning: ambiguous stereochemistry - linear bond arrangement - at atom 9 ignored
[15:22:38] Warning: ambiguous stereochemistry - linear bond arrangement - at atom 9 ignored
[15:22:38] Warning: ambiguous stereochemistry - linear bond arrangement -

MolGridWidget()

In [8]:
help(mols2grid.display)

Help on function display in module mols2grid.dispatch:

display(arg, **kwargs)
    Display molecules on an interactive grid.
    
    Parameters: Data
    ----------------
    arg : pandas.DataFrame, SDF file or list of molecules
        The input containing your molecules.
    smiles_col : str or None, default="SMILES"
        If a pandas dataframe is used, name of the column with SMILES.
    mol_col : str or None, default=None
        If a pandas dataframe is used, name of the column with RDKit molecules.
        If available, coordinates and atom/bonds annotations from this will be
        used for depiction.
    
    Parameters: Display
    -------------------
    template : str, default="interactive"
        Either ``"interactive"`` or ``"static"``. See ``render()`` for more details.
    size : tuple, default=(130, 90)
        The size of the drawing canvas. The cell minimum width is set to the
        width of the image, so if the cell padding is increased, the image will
       